In [2]:
import pandas as pd
import re

In [3]:
lts_list = ['0-7', '2-22', '3-22', '6-35', '7-24', '9-21', '11-22', '12-14', '12-26', '13-11', '13-19', '14-27', '15-3', '16-11']
df_list = []
df_by_category_list = []

for idx, lts in enumerate(lts_list):
    df = pd.read_pickle(f'../../data/test/lts-{lts}/lts-{lts}.df')
    df_list.append(df)

In [4]:
df_list[0].head(5)

,package,version,stability,cabal-file,categories,deps,provided-modules,src-dirs,main-modules,mtl-direct,version-range-deps
0,abstract-deque,0.3,[],C:/Users/nicol/Desktop/lts/lts-0-7\abstract-de...,[data],"[array, base, containers, random, time]","[Data.Concurrent.Deque.Class, Data.Concurrent....",[],[],0,"[(array, -any), (base, ==4.*), (containers, -a..."
1,abstract-par,0.3.3,[experimental],C:/Users/nicol/Desktop/lts/lts-0-7\abstract-pa...,"[control, parallelism, monads]","[base, deepseq]","[Control.Monad.Par.Class, Control.Monad.Par.Un...",[],[],0,"[(base, ==4.*), (deepseq, >=1.1)]"
2,AC-Vector,2.3.2,[experimental],C:/Users/nicol/Desktop/lts/lts-0-7\AC-Vector\A...,"[data, math, numerical, graphics]",[base],"[Data.BoundingBox.B1, Data.BoundingBox.B2, Dat...",[.],[],0,"[(base, ==4.*)]"
3,accelerate,0.15.0.0,[experimental],C:/Users/nicol/Desktop/lts/lts-0-7\accelerate\...,"[compilers/interpreters, concurrency, data, pa...","[array, base, containers, fclabels, ghc-prim, ...","[Data.Array.Accelerate, Data.Array.Accelerate....",[],[],0,"[(array, >=0.3), (base, ==4.7.*), (containers,..."
4,ace,0.6,[],C:/Users/nicol/Desktop/lts/lts-0-7\ace\ace-0.6...,[linguistics],"[attoparsec, base, blaze-html, blaze-markup, d...","[ACE, ACE.Combinators, ACE.Html, ACE.Parsers, ...",[src/],[],0,"[(attoparsec, >=0.10.4.0), (base, ==4.*), (bla..."


In [5]:
type(df_list[0]['version'])

pandas.core.series.Series

In [62]:
diff_version = {}
diff_version['hola'] = []
diff_version['hola'].append([5,5,'hola','uwu'])
diff_version['hola'].append([5,6,'hola','uwu'])

In [63]:
diff_version

{'hola': [[5, 5, 'hola', 'uwu'], [5, 6, 'hola', 'uwu']]}

In [78]:
diff_version = {}
for idx,row in df_list[0].iterrows():
    pkg = row['package']
    diff_version[pkg] = []
    for dep in row['version-range-deps']:
        dep_name = dep[0]
        pkg_version = str(df_list[0][df_list[0]['package']==dep_name]['version'])
        
        if dep_name != '' and dep_name != 'rts' and dep_name != 'hslua' and dep_name != 'QuickCheck':
            characters = '<>=*'
            version = dep[1]
            if not re.search('any',version):
                #clean the range version of deps
                for x in range(len(characters)):
                    version = version.replace(characters[x],'')            
                list_version = version.split(' ')
                size = len(list_version)
                
                #clean the version of pkg
                without_space = pkg_version.replace('Name: version, dtype: object:','').split(' ')
                pkg_version_clean = without_space[4].replace('\nName:','')
                list_version_pkg = pkg_version_clean.split('.')

                if size == 1:
                    if list_version[0][-1] == '.':
                        list_number = list_version[0].split('.')
                        value = list_number.pop()
                        count = 0      
                        for i in range(len(value)):
                            if value[i] == list_version_pkg[i]:
                                count +=1
                        if  count == len(value):
                            diff_version[pkg].append([dep_name,pkg_version_clean,dep[1],'normal'])
                        else:
                            diff_version[pkg].append([dep_name,pkg_version_clean,dep[1],'fuera de rango'])  
                    else:
                        count_normal = 0
                        list_split_version = list_version[0].split('.')
                        for i in range(len(list_split_version)):
                            if list_version_pkg[i] >= list_split_version[i]:
                                count_normal+=1
                        if  count_normal == len(list_split_version):
                            diff_version[pkg].append([dep_name,pkg_version_clean,dep[1],'normal'])
                        else:
                            diff_version[pkg].append([dep_name,pkg_version_clean,dep[1],'atrasada'])                 

                elif size == 3:
                    lower_ver = list_version[0].split('.')
                    upper_ver = list_version[2].split('.')
                    count_lower = 0
                    count_uper = 0
                    print(list_version_pkg,lower_ver)
                    for i in range(len(lower_ver)):
                        if list_version_pkg[i] >= lower_ver[i]:
                            count_lower+=1
                    for i in range(len(upper_ver)):
                        if list_version_pkg[i] < upper_ver[i]:
                            count_uper+=1
                    if count_lower == len(lower_ver) and count_uper == len(upper_ver):
                        diff_version[pkg].append([dep_name,pkg_version_clean,dep[1],'normal'])
                    elif count_lower < len(lower_ver):
                        diff_version[pkg].append([dep_name,pkg_version_clean,dep[1],'atrasada'])
                    elif count_uper < len(upper_ver):
                        diff_version[pkg].append([dep_name,pkg_version_clean,dep[1],'adelantada'])


['4', '7', '0', '1'] ['2']
['0', '5', '0', '0'] ['0', '3']
['4', '7', '0', '1'] ['4', '0']
['4', '2'] ['1', '2']
['0', '16', '0', '1'] ['0', '1']
['0', '5', '0', '0'] ['0', '2']
['4', '7', '0', '1'] ['4', '5']
['0', '5', '5', '1'] ['0', '2']
['4', '9'] ['4', '6', '1']


IndexError: list index out of range

In [69]:
df_list[3][df_list[3]['package'] =='abstract-deque']

,package,version,stability,cabal-file,categories,deps,provided-modules,src-dirs,main-modules,mtl-direct,version-range-deps
0,abstract-deque,0.3,[],C:/Users/nicol/Desktop/lts/lts-6-35\abstract-d...,[data],"[array, base, containers, random, time]","[Data.Concurrent.Deque.Class, Data.Concurrent....",[],[],0,"[(array, -any), (base, ==4.*), (containers, -a..."


In [75]:
if '0.10.12.2' < '0.7.1':
    print('uwu')

uwu


In [77]:
for pkg in diff_version.keys():
    for dep in diff_version[pkg]:
        state = dep[3]
        if state != 'normal':
            print('----------------------------------------')
            print('Package',pkg)
            print('Name_dep',dep[0])
            print('version_dep',dep[1])
            print('version_pkg',dep[2])
            print('State',dep[3])
            print('----------------------------------------')

----------------------------------------
Package ace
Name_dep attoparsec
version_dep 0.12.1.2
version_pkg >=0.10.4.0
State atrasada
----------------------------------------
----------------------------------------
Package ace
Name_dep blaze-html
version_dep 0.7.0.3
version_pkg >=0.5.1.0
State atrasada
----------------------------------------
----------------------------------------
Package ace
Name_dep text
version_dep 1.1.1.3
version_pkg >=0.11.2.0
State atrasada
----------------------------------------
----------------------------------------
Package active
Name_dep array
version_dep 0.5.0.0
version_pkg >=0.3 && <0.6
State adelantada
----------------------------------------
----------------------------------------
Package active
Name_dep base
version_dep 4.7.0.1
version_pkg >=4.0 && <4.8
State adelantada
----------------------------------------
----------------------------------------
Package active
Name_dep semigroupoids
version_dep 4.2
version_pkg >=1.2 && <4.3
State adelantada
---